In [ ]:
import re
from pathlib import Path
import pandas as pd
from openpyxl import load_workbook

# ==========================================================
# SCRIPT FINAL (1 MES) - 31 DÍAS SIEMPRE + ESCALA + IVA
# - Lee TXT (unicode) con auto encoding y auto separador
# - Filtra año/mes objetivo
# - Agrega por tienda + día (suma Cantidad y Total líneas)
# - Genera SIEMPRE 31 filas (días 1..31) por tienda (0 si no hay venta)
# - Pega valores en G12 (unidades) y H12 ($) hacia abajo
# - Mapea tiendas por celda A2 en cada hoja
# - Ignora tiendas/canales que no existan en el Excel (Ecom/ML)
# - Corrige $: Total líneas * 1000 * 1.19
# - Guarda copia en carpeta Resultado
# ==========================================================

# =========================
# RUTAS (AJUSTA SOLO ESTO)
# =========================
ORIGEN_DIR = Path(r"C:\Users\Jaime Valderrama\OneDrive - American Sportswear, S.A\Documentos\Jaime\Testing\Nueva carpeta\Origen")
TXT_NAME = "ventaspy.txt"

# Excel mensual a actualizar (Ej: Febrero: "VENTAS DIARIAS CK 02 2026.xlsx")
EXCEL_MONTHLY_NAME = "VENTAS DIARIAS CK 01 2026.xlsx"

OUTPUT_DIR = Path(r"C:\Users\Jaime Valderrama\OneDrive - American Sportswear, S.A\Documentos\Jaime\Testing\Nueva carpeta\Resultado\Ventas py")

# =========================
# PERIODO A CARGAR (AJUSTA)
# =========================
TARGET_YEAR = 2025
TARGET_MONTH = 2  # 1=Enero, 2=Febrero, etc.

# =========================
# FACTORES PARA $
# =========================
SALES_SCALE = 1000.0   # corrige el "dividido en 1000"
IVA_FACTOR = 1.19      # multiplica por 1,19

# =========================
# CONFIG EXCEL
# =========================
STORE_CELL = "A2"     # donde está el nombre del almacén dentro de cada hoja
START_ROW = 12        # G12/H12
COL_UNITS = 7         # G
COL_SALES = 8         # H

# =========================
# COLUMNAS DEL TXT
# =========================
COL_DATE  = "Fecha Documento"
COL_STORE = "Almacén"
COL_QTY   = "Cantidad"       # unidades
COL_AMT   = "Total líneas"   # $

# =========================
# HELPERS
# =========================
def norm(s: str) -> str:
    s = "" if s is None else str(s)
    s = s.strip()
    s = re.sub(r"\s+", " ", s)
    return s.lower()

def last_used_row_in_cols(ws, cols, start_row):
    last = start_row - 1
    for r in range(start_row, ws.max_row + 1):
        for c in cols:
            if ws.cell(r, c).value not in (None, ""):
                last = r
    return last

def clear_range(ws, row_from, row_to, cols):
    for r in range(row_from, row_to + 1):
        for c in cols:
            ws.cell(r, c).value = None

def try_read_txt(path: Path):
    """
    Intenta leer TXT probando encodings y separadores comunes.
    Devuelve (df, encoding_usado, sep_usado)
    """
    encodings = ["utf-8-sig", "utf-16", "latin-1"]
    seps = ["\t", ";", ",", "|"]
    last_err = None
    for enc in encodings:
        for sep in seps:
            try:
                df = pd.read_csv(path, sep=sep, encoding=enc, dtype=str, engine="python")
                if df.shape[1] <= 1:
                    continue
                return df, enc, sep
            except Exception as e:
                last_err = e
    raise RuntimeError(f"No pude leer el TXT con combinaciones comunes. Último error: {last_err}")

def to_number(series: pd.Series) -> pd.Series:
    """
    Convierte a número manejando coma decimal.
    """
    return pd.to_numeric(series.astype(str).str.replace(",", ".", regex=False), errors="coerce").fillna(0)

# =========================
# PATHS
# =========================
txt_path = ORIGEN_DIR / TXT_NAME
excel_path = ORIGEN_DIR / EXCEL_MONTHLY_NAME

if not ORIGEN_DIR.exists():
    raise FileNotFoundError(f"No existe ORIGEN_DIR: {ORIGEN_DIR}")
if not txt_path.exists():
    raise FileNotFoundError(f"No encuentro TXT: {txt_path}")
if not excel_path.exists():
    raise FileNotFoundError(f"No encuentro Excel mensual: {excel_path}")

# =========================
# 1) LEER TXT
# =========================
df, used_enc, used_sep = try_read_txt(txt_path)
print(f"TXT leído OK | encoding={used_enc} | sep={'TAB' if used_sep == chr(9) else used_sep}")
print("Columnas leídas:", list(df.columns))
print("Filas:", len(df))

required = [COL_DATE, COL_STORE, COL_QTY, COL_AMT]
missing = [c for c in required if c not in df.columns]
if missing:
    raise ValueError(f"Faltan columnas requeridas: {missing}")

# Parse fecha y números
df[COL_DATE] = pd.to_datetime(df[COL_DATE], errors="coerce", dayfirst=True)
bad_dates = df[df[COL_DATE].isna()]
if len(bad_dates) > 0:
    raise ValueError(f"Hay {len(bad_dates)} filas con fecha inválida en '{COL_DATE}'.")

df[COL_QTY] = to_number(df[COL_QTY])
df[COL_AMT] = to_number(df[COL_AMT])

df["Año"] = df[COL_DATE].dt.year.astype(int)
df["Mes"] = df[COL_DATE].dt.month.astype(int)
df["Día"] = df[COL_DATE].dt.day.astype(int)

# =========================
# 2) FILTRAR PERIODO
# =========================
df = df[(df["Año"] == TARGET_YEAR) & (df["Mes"] == TARGET_MONTH)].copy()
print(f"Filas tras filtro {TARGET_YEAR}-{TARGET_MONTH:02d}:", len(df))
if df.empty:
    raise ValueError(f"No hay datos para {TARGET_YEAR}-{TARGET_MONTH:02d} en el TXT.")

# =========================
# 3) AGREGAR POR TIENDA + DÍA
# =========================
df_day = (
    df.groupby([COL_STORE, "Día"], as_index=False)[[COL_QTY, COL_AMT]]
      .sum()
)

# =========================
# 4) EXPANDIR A 31 DÍAS (SIEMPRE) CON CEROS
# =========================
all_days = pd.DataFrame({"Día": list(range(1, 32))})
stores = pd.DataFrame({COL_STORE: df[COL_STORE].dropna().unique()})
grid = stores.merge(all_days, how="cross")

df_day = (
    grid.merge(df_day, on=[COL_STORE, "Día"], how="left")
        .fillna({COL_QTY: 0, COL_AMT: 0})
        .sort_values([COL_STORE, "Día"])
)

print("Registros a pegar (tienda*día):", len(df_day), " (31 * #tiendas_con_datos)")

# =========================
# 5) ABRIR EXCEL Y MAPEAR HOJAS POR A2
# =========================
wb = load_workbook(excel_path)

store_to_ws = {}
for ws in wb.worksheets:
    store_val = ws[STORE_CELL].value
    if store_val is not None and str(store_val).strip() != "":
        store_to_ws[norm(store_val)] = ws

print(f"Hojas mapeadas por {STORE_CELL}:", len(store_to_ws))

# =========================
# 6) PEGAR EN G12/H12 (31 filas) + $ * escala * IVA
# =========================
updated = 0
warnings = 0

for store_name, df_store in df_day.groupby(COL_STORE, sort=False):
    ws = store_to_ws.get(norm(store_name))
    if ws is None:
        # Ecommerce / MercadoLibre u otros que no están en el excel: se ignoran
        print(f"[WARN] No existe hoja en Excel para '{store_name}' (se omite)")
        warnings += 1
        continue

    df_store = df_store.sort_values("Día")

    # Unidades tal cual
    units = df_store[COL_QTY].astype(float).tolist()

    # $ corregido: escala * IVA
    sales = (df_store[COL_AMT].astype(float) * SALES_SCALE * IVA_FACTOR).tolist()

    # Asegurar 31 filas
    units = (units + [0.0]*31)[:31]
    sales = (sales + [0.0]*31)[:31]
    n = 31

    # limpiar desde fila 12 hasta lo que esté usado (por si había más)
    last_used = last_used_row_in_cols(ws, cols=[COL_UNITS, COL_SALES], start_row=START_ROW)
    target_last = START_ROW + n - 1
    clear_to = max(last_used, target_last)
    if clear_to >= START_ROW:
        clear_range(ws, START_ROW, clear_to, cols=[COL_UNITS, COL_SALES])

    # pegar valores
    for i in range(n):
        r = START_ROW + i
        ws.cell(r, COL_UNITS).value = units[i]
        ws.cell(r, COL_SALES).value = sales[i]

    print(f"OK | {ws.title} | {STORE_CELL}='{ws[STORE_CELL].value}' | 31 filas -> G{START_ROW}:H{START_ROW+n-1}")
    updated += 1

# =========================
# 7) GUARDAR OUTPUT (COPIA)
# =========================
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
out_file = OUTPUT_DIR / f"{excel_path.stem}_ACTUALIZADO.xlsx"

print("Guardando archivo...", flush=True)
wb.save(out_file)
print("Guardado OK.", flush=True)

print("\nResumen:")
print(f"- Hojas actualizadas: {updated}")
print(f"- Tiendas omitidas (sin hoja): {warnings}")
print(f"- Archivo generado: {out_file}")


TXT leído OK | encoding=utf-16 | sep=TAB
Columnas leídas: ['Fecha Documento', 'Código de almacén', 'Almacén', 'Cantidad', 'Total líneas']
Filas: 933028
Filas tras filtro 2025-01: 84412
Registros a pegar (tienda*día): 651  (31 * #tiendas_con_datos)
Hojas mapeadas por A2: 19
OK | ALTO LAS CONDES | A2='CK - Alto Las Condes' | 31 filas -> G12:H42
OK | ANTOFAGASTA | A2='CK - Antofagasta' | 31 filas -> G12:H42
OK | BUENAVENTURA | A2='CK - Buenaventura' | 31 filas -> G12:H42
OK | CHILLAN | A2='CK - Chillán' | 31 filas -> G12:H42
OK | COQUIMBO | A2='CK - Coquimbo' | 31 filas -> G12:H42
OK | COSTANERA | A2='CK - Costanera' | 31 filas -> G12:H42
OK | CURAUMA | A2='CK - Curauma' | 31 filas -> G12:H42
OK | EASTON CENTER | A2='CK - Easton' | 31 filas -> G12:H42
[WARN] No existe hoja en Excel para 'CK - Ecommerce' (se omite)
OK | FLORIDA | A2='CK - Florida Center' | 31 filas -> G12:H42
OK | LA SERENA | A2='CK - La Serena' | 31 filas -> G12:H42
OK | MARINA | A2='CK - Marina' | 31 filas -> G12:H42
OK 